# Object Detection with TensorFlow Object Detection API

## Learning Objectives 
- Learn how to setup the TensorFlow Object Detection API environment
- Learn how to preprocess datasets for object detection
- Learn how to restore pre-trained weights for transfer learning
- Learn how to run transfer learning 
- Learn how to use trained models and visualize the results.

In this lab, you will learn how to apply transfer learning using [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection).

If we build an object detection program from scratch, we need to write a complex model, preprocessing and postprocessing codes, and multiple loss functions.<br>
TensorFlow Object Detection API provides useful tools to speed up the building process.

In this notebook, we will retrieve a pre-trained model architecture and its weights and run transfer learning using our own dataset.

## Environment Setup

We can install Object Detection API directly, but here let's create a separate kernel environment to avoid affecting the other notebook execution.

The script is prepared in [asl-ml-immersion/kernels/object_detection.sh](../../../kernels/object_detection.sh), and you can build the kernel from the `make` command below.

This command executes the following operations:
- Setup virtual env and create a new Jupyter Notebook kernel
- Download the Object Detection API module from GitHub
- Install the necessary dependencies of the Object Detection API

In [ ]:
!cd ~/asl-ml-immersion && make object_detection_kernel

After running the command above, change the kernel of this notebook to `object_detection_kernel` and continue to run from the next cell.

1. Select the kernel section on the top. (`Python 3` is the default kernel name)
> <img width="1039" alt="image" src="https://user-images.githubusercontent.com/6895245/196198001-681184a4-a289-4352-b137-f66697673d55.png">

2. Select `object_detection_kernel`
> <img width="316" alt="image" src="https://user-images.githubusercontent.com/6895245/196198300-4447a928-52c5-41a3-b3a6-63265b5791a9.png">


## Import Libraries
Note that we import some modules from Object Detection API.

In [ ]:
import io
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

## Data Preprocessing
Here let's create a data loader for [`salad.csv`](../dataset_import_files/salads.csv) dataset which is created for AutoML Object Detection.

In this dataset, all the rows are organized in `path, label, x_min, y_min, , ,x_max,y_max,,`
- `path`: the path to the image file
- `label`: a label for a bounding box
- `x_min`, `y_min`, `x_max`, `y_max`: coordinates of the boundig box rectangle.

In [ ]:
CSV_PATH = "../dataset_import_files/salads.csv"
!head $CSV_PATH

Note that each image has multiple rows for multiple bouding boxes. In order to use this dataset in Object Detection API, we will need to preprocess this and aggregate them later.

First, we create unique image list (`PATHS`) and unique class labels (`CLASSIS`) using pandas dataframe. And then define lookup tables with [`StringLookup`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup) module. 

In [ ]:
df = pd.read_csv(CSV_PATH)
PATHS = df.iloc[:, 0].unique()
CLASSES = df.iloc[:, 1].unique()

In [ ]:
img_id_lookup = tf.keras.layers.StringLookup(vocabulary=PATHS)
class_lookup = tf.keras.layers.StringLookup(vocabulary=CLASSES)

### Read CSV and create dataset

Next, we define tf.data pipeline to:
1. Load and parse CSV data
2. Aggregate multiple rows (bouning boxes) into lists by image paths as keys
3. Load the image path and decode the jpeg file into Tensor
4. Split data into training and test dataset

#### 1. Load and Parse CSV data

**Exercise**: Complete the cell below by adding the code to:
- one-hot encode labels using `class_lookup` defined above.
- extract the bounding box locations in `[y_min, x_min, y_max, x_max]` order from `row` defined in `[path, label, x_min, y_min, , ,x_max,y_max,,]` using `tf.gather()` referring to the [document](https://www.tensorflow.org/api_docs/python/tf/gather). Also, cast the data type from string to integer using `tf.string.to_number()`

In [ ]:
def parse_csv(row):
    row = tf.strings.split(row, ",")
    path = row[0]
    label = row[1]
    # TODO: One-hot Encode labels.
    # one_hot_label =

    # TODO: Extract bounding box location in [y_min, x_min, y_max, x_max] order.
    # bbox =
    return path, one_hot_label, bbox


data = tf.data.TextLineDataset(CSV_PATH).map(parse_csv)

#### 2. Aggregate bounding boxes
Group multiple rows by using image paths as keys. 

We can use [`Dataset.group_by_window`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#group_by_window) to achieve this.

For this method, we need to specify which feature we use as aggregation key (`key_func`), how to aggregate other features (`reduce_func`), and the max window size to aggregate (`window_size`).<br>
In this `salad.csv` case:
- Key: ID of Paths
- Aggregation: combine in lists (`Dataset.batch`)
- Window Size: the max number of the bounding boxes in one image. 50 should be enough for this dataset as no image has more than 50 object.

In [ ]:
WINDOW_SIZE = 50
SEED = 3

data = data.group_by_window(
    key_func=lambda path, label, bbox: img_id_lookup(path),
    reduce_func=lambda _, window: window.batch(WINDOW_SIZE),
    window_size=WINDOW_SIZE,
).shuffle(len(PATHS), seed=SEED)

#### 3. Load image path and decode jpeg file
Define image loader and decoder.

In [ ]:
def decode_jpg(path, label, bbox):
    img = tf.io.read_file(path[0])
    img = tf.io.decode_jpeg(img, channels=3)
    return path[0], img, label, bbox


data = data.map(decode_jpg)

#### 4. Split data
Split the preprocessed data into training (80%) and test (20%) datasets.

In [ ]:
TRAIN_SIZE = int(len(PATHS) * 0.8)

train_data = data.take(TRAIN_SIZE)
test_data = data.skip(TRAIN_SIZE)

Let's retrieve and take a look at the preprocessing result.

In [ ]:
for path, img, label, bbox in train_data.take(1):
    print(f"path: {path}")
    print(f"img shape: {img.shape}")
    print(f"label (index): {label}")
    print(f"bbox (x_min, y_min, x_max, y_max): {bbox}")

    print("*" * 50)
    # Check the number of bounding boxes.
    assert label.shape[0] == bbox.shape[0]
    print(f"This image has {label.shape[0]} bounding boxes")

Now we can see multiple bounding boxes and labels are successfully linked to one image.

## Visualize Training Data
Let's visualize the preprocessed dataset using Object Detection API helper functions.

In [ ]:
def plot_detections(image_np, boxes, classes, scores, category_index):

    image_np_with_annotations = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.4,
        line_thickness=20,
    )

    plt.imshow(image_np_with_annotations)

This helper function requires the class mapping dictionary to assign different colors to each bounding box.

In [ ]:
category_index = {
    i: {"id": i, "name": class_name}
    for i, class_name in enumerate(class_lookup.get_vocabulary())
    if class_name != "[UNK]"
}
category_index

Now we can call this function and check some samples from the training dataset.<br>
We assign `1` (100%) for the confidence scores as the bounding boxes are ground truth labels.

In [ ]:
plt.figure(figsize=(30, 15))
for idx, (path, img, label, bbox) in enumerate(train_data.take(5)):
    plt.subplot(2, 3, idx + 1)
    plot_detections(
        img.numpy(),
        bbox.numpy(),
        label.numpy(),
        [1] * label.shape[0],  # give boxes a score of 100%
        category_index,
    )
plt.show()

Though the ground truth quality is not perfect (due to a poor labeling process), now we can see the data preprocessing is successfully done, and bounding boxes are shown around objects.

If you can not see any bounding boxes in some images, that is because only one bounding box for entire images (i.e., `[0,0,1,1]`) is assigned for the images, and it is not appropriately visualized.

## Object Detection Model
Now our data is ready. Let's build an object detection model to train.<br>
TensorFlow Object Detection API provides many useful tools, including [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md), a collection of detection models pre-trained on [the COCO 2017 dataset](https://cocodataset.org/#home).

In this lab, we use a single stage detection with Resnet 50 v1 FPN feature extractor (a.k.a. RetinaNet) designed for 640x640 sized images. Under the Model Zoo repository, it is named as`"SSD ResNet50 V1 FPN 640x640 (RetinaNet50)"`

RetinaNet paper: https://arxiv.org/abs/1708.02002

### Download Pre-trained Weights
Let's retrieve the weights 
From [the Model Zoo page](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md), you can check the URL of each pre-trained weight file in tar.gz format (Right click the model name -> Copy Link Address). 

In [ ]:
WEIGHT_FILE_PREFIX = (
    "http://download.tensorflow.org/models/object_detection/tf2/20200711"
)
MODEL_NAME = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"

# Download the weights file
!wget {WEIGHT_FILE_PREFIX}/{MODEL_NAME}.tar.gz
!tar -xf{MODEL_NAME}.tar.gz

### Setup a Model
We don't need to set up the model from scratch. Instead, we only need to load a [config file](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2) for the specific model from the Object Detection API repository.

In this case, we can use `ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config`.

In [ ]:
# tf.keras.backend.clear_session()

pipeline_config = (
    f"models/research/object_detection/configs/tf2/{MODEL_NAME}.config"
)
!head -30 $pipeline_config

Let's load pipeline config using `config_util.get_configs_from_pipeline_file()` and modify a little.

Since we are working off of a [COCO](https://cocodataset.org/#home) architecture which predicts 90 class slots by default, we override the `num_classes` field here to be just 5 (for our new salad datasets).

Then, we can build a model with `model_builder.build()`.

**Exercise**: Modify the config to update the number of classes.<br>

**Hint**: You can find the default value in the `num_classes` key above.

In [ ]:
configs = config_util.get_configs_from_pipeline_file(pipeline_config)

model_config = configs["model"]
# TODO: update the number of classes.

detection_model = model_builder.build(
    model_config=model_config, is_training=True
)

Run the model through a dummy image so that variables are created.

In [ ]:
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)

## Training

### Restore Pretrained Weights
Instead of training `detection_model` from scratch, we restore weights from the pre-trained checkpoint file and run transfer learning.

Let's set up object-based checkpoint restore using `tf.train.Checkpoint` where we can specify the objects we want to checkpoint as attributes on the object. <br>

At first, we restore the 'heads' of the detection model. RetinaNet has two prediction 'heads' --- one for classification and the other for box regression.<br>
We will restore the box regression head but initialize the classification head from scratch (we show the omission below by commenting out the line that we would add if we wanted to restore both heads)

In [ ]:
checkpoint_path = f"{MODEL_NAME}/checkpoint/ckpt-0"

box_predictor_ckpt = tf.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
)

Next, we set up another checkpoint object for the entire model and restore all variables.

In order to silence warnings about incomplete checkpoint restores, we also add `expect_partial()` when we call `tf.train.Checkpoint.restore()`.

In [ ]:
model_ckpt = tf.train.Checkpoint(
    _feature_extractor=detection_model._feature_extractor,
    _box_predictor=box_predictor_ckpt,
)

ckpt = tf.train.Checkpoint(model=model_ckpt)
ckpt.restore(checkpoint_path).expect_partial()

### Transfer Learning

In transfer learning, we reuse the knowledge our model learned from pretraining and update the heads only.

To specify which variable should/should not be updated, let's create a list of the variables for the two detection heads (`to_fine_tune`).<br>

In [ ]:
trainable_variables = detection_model.trainable_variables

to_fine_tune = []

prefixes_to_train = [
    "WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead",
    "WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead",
]

for var in trainable_variables:
    if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
        to_fine_tune.append(var)

Now let's define the training loop that will:
- provide ground truth label data to the model using `provide_groudtruth` to compute loss value later
- call `model.predict` and get the result dictionary
- call `model.loss` to compute loss values and sum up box regression loss (`localization_loss`) and classification loss (`classification_loss`)
- update the targeted variables using the computed loss value

RetinaNet uses the standard smooth L1 loss for box regression task and focal loss for classification task.<br>
For details, please refer to the [original paper](https://arxiv.org/abs/1708.02002).

**Exercise**: Define the total_loss in the cell below. You can access each loss in `losses_dict["Loss/<loss name>"]`.

In [ ]:
def train_step_fn(
    image_tensors,
    groundtruth_boxes_list,
    groundtruth_classes_list,
    batch_size,
):

    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)

    detection_model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list,
    )

    with tf.GradientTape() as tape:
        prediction_dict = detection_model.predict(image_tensors, shapes)
        losses_dict = detection_model.loss(prediction_dict, shapes)
        # TODO: Define Total Loss to minimize.
        # total_loss =
        gradients = tape.gradient(total_loss, to_fine_tune)
        optimizer.apply_gradients(zip(gradients, to_fine_tune))
    return total_loss

Before running training, let's apply the preprocessing for training data like resizing and rescaling.<br>
Instead of writing the logic by ourselves, we can use the predefined `model.preprocess()` via Object Detection API.

In [ ]:
def _preprocess(path, img, label, bbox):
    label = tf.one_hot(label, len(CLASSES))

    img = tf.expand_dims(img, axis=0)
    img = tf.cast(img, tf.float32)
    img = detection_model.preprocess(img)[0]
    img = tf.squeeze(img, axis=0)
    return path, img, label, bbox


preprocessed_train_data = train_data.map(_preprocess)

Now let's iterate over training data and update the selected weights.

This will take 20 minutes with one NVIDIA T4 GPU.

In [ ]:
NUM_SAMPLES = TRAIN_SIZE
BATCH_SIZE = 1
EPOCHS = 20
LEARNING_RATE = 0.001

optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

print("Start fine-tuning!", flush=True)

total_loss = []
for idx, (path, img, label, bbox) in enumerate(
    preprocessed_train_data.batch(BATCH_SIZE).repeat(EPOCHS)
):
    progress_percent = (idx % NUM_SAMPLES) / NUM_SAMPLES * 100

    total_loss.append(train_step_fn(img, bbox, label, BATCH_SIZE))

    if idx != 0:
        if idx % NUM_SAMPLES == 0:
            print("")
        else:
            current_epoch = idx // NUM_SAMPLES + 1
            current_loss = np.mean(total_loss[idx - (idx % NUM_SAMPLES) : idx])
            print(
                (
                    f"epoch {current_epoch:<2}/{EPOCHS},"
                    f"loss: {current_loss:.4f},"
                    f"[{'■' * (math.ceil(progress_percent) // 5): <20}]"
                    f"{math.ceil(progress_percent)}%"
                ),
                end="\r",
                flush=True,
            )

print("")
print("Done fine-tuning!")

## Detection
Let's try to detect bounding boxes over the test data, and check if the transfer learning went well.

In object detection, we need to apply complex postprocessing, including non-max suppression (NMS), to the prediction outputs.<br>
But you don't need to implement it from scratch. The Object Detection has a predefined `model.postprocess` method we can easily use.

In [ ]:
plt.figure(figsize=(30, 15))

for idx, (path, img, label, bbox) in enumerate(test_data.take(5)):
    plt.subplot(2, 3, idx + 1)

    _, preprocessed_img, _, _ = _preprocess(path, img, label, bbox)
    prediction_dict = detection_model.predict(
        tf.expand_dims(preprocessed_img, axis=0), shapes
    )
    detections = detection_model.postprocess(prediction_dict, shapes)

    plot_detections(
        img.numpy(),
        detections["detection_boxes"][0].numpy(),
        detections["detection_classes"][0].numpy().astype(np.uint32),
        detections["detection_scores"][0].numpy(),
        category_index,
    )

plt.show()

Still not perfect, but we can see our model is tuned toward our own salad dataset successfully.

Here are a few ideas to improve the performance:
- Improve the dataset label quality
- Add more data
- Tune learning rate and scheduling strategy
- Train longer time

## Summary
In this notebook, we learned how to build and train an object detection model using the Tensorflow Object Detection API.


## License

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License